In [25]:
import cv2
import glob
import numpy as np
import pandas as pd

In [16]:
data = pd.read_pickle("w_arraydef_otrodataset.pkl")

In [18]:
data.head()

,pose,png,path,array
0,chaturanga dandasana,33-0.png,../img/chaturanga dandasana/33-0.png,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
1,chaturanga dandasana,76-0.png,../img/chaturanga dandasana/76-0.png,"[[244, 244, 244, 244, 244, 243, 244, 244, 244,..."
2,chaturanga dandasana,50-2.png,../img/chaturanga dandasana/50-2.png,"[[254, 255, 255, 175, 122, 145, 143, 146, 144,..."
3,chaturanga dandasana,9-0.png,../img/chaturanga dandasana/9-0.png,"[[246, 241, 247, 246, 247, 226, 219, 226, 224,..."
4,chaturanga dandasana,92-0.png,../img/chaturanga dandasana/92-0.png,"[[196, 190, 198, 196, 198, 197, 198, 200, 196,..."


In [20]:
new_dummies = pd.get_dummies(data['pose'])

In [21]:
data_new = data.join(new_dummies)
data_new.head()

,pose,png,path,array,ardha matsyendrasana,chaturanga dandasana,garudasana,other
0,chaturanga dandasana,33-0.png,../img/chaturanga dandasana/33-0.png,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",0,1,0,0
1,chaturanga dandasana,76-0.png,../img/chaturanga dandasana/76-0.png,"[[244, 244, 244, 244, 244, 243, 244, 244, 244,...",0,1,0,0
2,chaturanga dandasana,50-2.png,../img/chaturanga dandasana/50-2.png,"[[254, 255, 255, 175, 122, 145, 143, 146, 144,...",0,1,0,0
3,chaturanga dandasana,9-0.png,../img/chaturanga dandasana/9-0.png,"[[246, 241, 247, 246, 247, 226, 219, 226, 224,...",0,1,0,0
4,chaturanga dandasana,92-0.png,../img/chaturanga dandasana/92-0.png,"[[196, 190, 198, 196, 198, 197, 198, 200, 196,...",0,1,0,0


In [23]:
data_new['gt'] = data_new[['ardha matsyendrasana','chaturanga dandasana','garudasana','other']].apply(lambda x: pd.Series([x.values]), axis=1)
data_new.head()

,pose,png,path,array,ardha matsyendrasana,chaturanga dandasana,garudasana,other,gt
0,chaturanga dandasana,33-0.png,../img/chaturanga dandasana/33-0.png,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",0,1,0,0,"[0, 1, 0, 0]"
1,chaturanga dandasana,76-0.png,../img/chaturanga dandasana/76-0.png,"[[244, 244, 244, 244, 244, 243, 244, 244, 244,...",0,1,0,0,"[0, 1, 0, 0]"
2,chaturanga dandasana,50-2.png,../img/chaturanga dandasana/50-2.png,"[[254, 255, 255, 175, 122, 145, 143, 146, 144,...",0,1,0,0,"[0, 1, 0, 0]"
3,chaturanga dandasana,9-0.png,../img/chaturanga dandasana/9-0.png,"[[246, 241, 247, 246, 247, 226, 219, 226, 224,...",0,1,0,0,"[0, 1, 0, 0]"
4,chaturanga dandasana,92-0.png,../img/chaturanga dandasana/92-0.png,"[[196, 190, 198, 196, 198, 197, 198, 200, 196,...",0,1,0,0,"[0, 1, 0, 0]"


In [19]:
data.shape

(339, 4)

In [26]:
X = np.stack(data['array'])
X = X / 255.0
y = np.stack(data_new['gt'])

In [27]:
img_rows, img_cols = 224, 224

In [30]:
ROOT_PATH = "../img/"
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
IMAGE_CHANNEL = 3

# Change brightness levels
def random_brightness(image):
    # Convert 2 HSV colorspace from BGR colorspace
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Generate new random brightness
    rand = random.uniform(0.3, 1.0)
    hsv[:, :, 2] = rand*hsv[:, :, 2]
    # Convert back to BGR colorspace
    new_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return new_img

# Zoom-in
def zoom(image):
    zoom_pix = random.randint(0, 10)
    zoom_factor = 1 + (2*zoom_pix)/IMAGE_HEIGHT
    image = cv2.resize(image, None, fx=zoom_factor,
                       fy=zoom_factor, interpolation=cv2.INTER_LINEAR)
    top_crop = (image.shape[0] - IMAGE_HEIGHT)//2
    left_crop = (image.shape[1] - IMAGE_WIDTH)//2
    image = image[top_crop: top_crop+IMAGE_HEIGHT,
                  left_crop: left_crop+IMAGE_WIDTH]
    return image

# fuction to read image from file and perform augmentations
def get_image(index, data, should_augment):
    # Read image and appropiate traffic light color
    image = cv2.imread(os.path.join(
        ROOT_PATH, data['path'].values[index].strip()))
    color = data['class'].values[index]
    should_chng_brightness = random.randint(0, 1)
    should_flip = random.randint(0, 1)
    should_zoom = random.randint(0, 1)
    if should_augment:
        if should_chng_brightness == 1:
            image = random_brightness(image)
        if should_flip == 1:
            image = cv2.flip(image, 1)
        if should_zoom == 1:
            image = zoom(image)

    return [image, color]

In [31]:
random_brightness(X)

error: OpenCV(4.2.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<3, -1, -1>, cv::impl::(anonymous namespace)::Set<0, 5, -1>, cv::impl::(anonymous namespace)::SizePolicy::NONE>::CvtHelper(cv::InputArray, cv::OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<3, -1, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 5, -1>, sizePolicy = cv::impl::(anonymous namespace)::SizePolicy::NONE]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 224
